# 1ST PART

In [1]:
#imports
from selenium import webdriver 
from selenium.webdriver.chrome.service import Service as ChromeService 
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.selenium_manager import SeleniumManager 
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from time import sleep
from googletrans import Translator
from google_trans_new import google_translator  
from deep_translator import GoogleTranslator
import os
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import pandas as pd
from sqlalchemy import create_engine
import warnings
import sys
from pyspark.sql import SparkSession
import pyspark.sql.functions as sf
# from pyspark.sql.functions import col,udf,lit,split
from pyspark.sql.types import IntegerType, StringType
from tqdm import tqdm
import pyspark.pandas as ps
import pandas as pd
import time
import os
import urllib.request
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

import warnings
warnings.filterwarnings("ignore")

from tqdm import tqdm
import pdfplumber

import subprocess
import pathlib
import ghostscript as gs

warnings.filterwarnings('ignore')

c:\Users\ayush\anaconda3\envs\morph\lib\site-packages\pyspark\pandas\__init__.py:50: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(


In [2]:
#Functions
def translator(s,source):
    return GoogleTranslator(source=source, target='en').translate(s)


def translator_v2(s,source='auto'):
        translator = Translator()
        return translator.translate(s, src='hi',dest='en').text if s != '' else 'null' 

translate_udf = sf.udf(translator_v2,StringType())

In [3]:
#selenium webdriver
input_params_dropdown_by_id = {}
input_params_dropdown_by_id['dbselect'] = '2023' #Select Year
input_params_dropdown_by_id['district_id'] = 'मुंबई उपनगर' #District
input_params_dropdown_by_id['taluka_id'] = 'अंधेरी' #Taluka
input_params_dropdown_by_id['village_id'] = 'बांद्रा' #Village

input_params_text_by_id = {}
input_params_text_by_id['free_text'] = '2023' 

url = 'https://pay2igr.igrmaharashtra.gov.in/eDisplay/Propertydetails/index' 
driver = webdriver.Chrome() 
driver.implicitly_wait(10)
driver.get(url)

for key, value in input_params_dropdown_by_id.items():
	print(key,value)
	dropdown = driver.find_element(by=By.ID,value=key)
	dropdown_select = Select(dropdown)
	for option in dropdown_select.options:
		if option.text == value:
			option.click()
			print(f'Clicked {option.text} for {key}')
			break
	driver.implicitly_wait(2)

#Input reg year
driver.find_element(by=By.ID, value='free_text').send_keys(input_params_text_by_id['free_text'])
driver.implicitly_wait(2)

#Input Captcha
captcha = input()
driver.find_element(by=By.ID, value='cpatchaTextBox').send_keys(captcha)
print(f'Entered Captcha {captcha}')
driver.implicitly_wait(2)

#Submit to get result set
driver.find_element(by=By.ID,value='submit').click()
driver.implicitly_wait(10)


#Click on 50 Pages
dropdown = driver.find_element(by=By.NAME, value='tableparty_length')
dropdown_select = Select(dropdown)
for option in dropdown_select.options:
	if option.text == 'All':
		option.click()
		print('Set to All Records')
		break

print('So Far, So Good')

dbselect 2023
Clicked 2023 for dbselect
district_id मुंबई उपनगर
Clicked मुंबई उपनगर for district_id
taluka_id अंधेरी
Clicked अंधेरी for taluka_id
village_id बांद्रा
Clicked बांद्रा for village_id
Entered Captcha 9XVW1
Set to All Records
So Far, So Good


In [4]:
#scrap records
records_raw = pd.DataFrame(columns=['अनु क्र.','दस्त क्र.','दस्त प्रकार','दू. नि. कार्यालय','वर्ष','लिहून देणार','लिहून घेणार','इतर माहीती','सूची क्र. २'])

print('Data Loading...')
for index, table in enumerate(tqdm(driver.find_elements(by=By.ID, value='tbdata'))):
    data = [item.text if item.text != 'सूची क्र. २' else item.find_element(by=By.TAG_NAME,value='a').get_attribute('href') for item in table.find_elements(by=By.XPATH, value=".//*[self::td or self::th]")]
    records_raw.loc[len(records_raw)] = data
    # print(data)
    # print(f'{index} Row appended',)

print('Finished!')
records_raw
records_translate = records_raw.copy()

Data Loading...


100%|██████████| 1209/1209 [03:17<00:00,  6.11it/s]

Finished!


In [5]:
# Insert Scraped Raw Data in postgres
db_user = 'postgres'
db_password = 'Sunrise12345'
db_host = '127.0.0.1'
db_port = '5432'
db_name = 'propReturns'

engine = create_engine(f'postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}')
table_name = 'record_details_raw'

records_raw.to_sql(table_name, engine, if_exists='replace', index=True)

print(f"{len(records_raw)} DataFrame has been inserted into the '{table_name}' table.")

1209 DataFrame has been inserted into the 'record_details_raw' table.


In [6]:
records_translate = records_raw.copy()
records_translate.columns = [translator(col,source='auto') for col in records_translate.columns]
cols_to_translate = [
    'diarrhea type',
    'Du. Prohibit. Office',
    'Will write', 
    'Will write down', 
    'Other information'
    ]

In [7]:
records_translate.columns

Index(['Sl no.', 'Diarrhea no.', 'diarrhea type', 'Du. Prohibit. Office',
       'Year', 'Will write', 'Will write down', 'Other information',
       'List no. 2'],
      dtype='object')

In [8]:
records_translate['to_translate'] = records_translate['diarrhea type'] + '__$__' \
                                    + records_translate['Du. Prohibit. Office'] + '__$__' \
                                    + records_translate['Will write'] + '__$__' \
                                    + records_translate['Will write down'] + '__$__' \
                                    + records_translate['Other information']  

In [9]:
records_translated = pd.DataFrame(columns=records_translate.columns)
batch_size = 1
start = 0
for batch in tqdm(range(start,len(records_translate))):
    # print(f'Batch -> [{batch},{batch+batch_size}]')
    temp_df = records_translate.iloc[batch:batch+batch_size,:].copy()
    temp_df['to_translate'] = temp_df['to_translate'].apply(translator_v2)
    # for col in cols_to_translate:
    #     # print(f'Translating {col}')
    #     temp_df[col] = temp_df[col].apply(lambda x: translator(x,source='hi'))
    #     # print(f'Done')
    records_translated = pd.concat([records_translated,temp_df])
records_translated.rename(columns={'Will write':'buyer_name','Will write down':'seller_name'},inplace=True)

 85%|████████▌ | 1033/1209 [04:45<00:48,  3.61it/s]


ReadTimeout: The read operation timed out

In [42]:
records_translated_og = records_translated.copy()

In [43]:
cols_to_translate = [
    'diarrhea type',
    'Du. Prohibit. Office',
    'buyer_name', 
    'seller_name', 
    'Other information'
    ]

records_translated[cols_to_translate] = records_translated['to_translate'].apply(lambda x: pd.Series(x.split('__$__')))

In [44]:
records_translated_og

,index,Sl no.,Diarrhea no.,diarrhea type,Du. Prohibit. Office,Year,buyer_name,seller_name,Other information,List no. 2,to_translate
0,0,1,13217,Rent lease,cum d.n. Andheri 7,25/07/2023,"1) Bhavin Sheth, Authorized Trustee on behalf ...","1) Hanmant Dhanvare, Executive Engineer, Mahar...",1) Other information: Piece or parcel of land ...,https://pay2igr.igrmaharashtra.gov.in/eDisplay...,Rent lease__$__cum d.n. Andheri 7__$__1) Bhavi...
1,1,2,4286,Development Agreement,Co. Andheri 7,10/03/2023,1) Ajit Singh Kartar Singh Chandok,"1) Paresh Ranchod Patel, Partner, Navish Realt...","1) Other information: Land and construction, p...",https://pay2igr.igrmaharashtra.gov.in/eDisplay...,Development Agreement__$__Co. Andheri 7__$__1)...
2,2,3,2449,Sale Deed,Co Dn. Andheri 7,09/02/2023,1) Nupur Anil Kalke Mukhtyar Shaila Anil Kalke...,1) Harsh Satpal Malhotra,"1) Other Information: House No: 701, Floor No:...",https://pay2igr.igrmaharashtra.gov.in/eDisplay...,Sale Deed__$__Co Dn. Andheri 7__$__1) Nupur An...
3,3,4,8691,66-Notice of Lis Pendency,cum. Andheri 7,22/05/2023,1) Pradeep Soni,,1) Other Information: City Civil Court at Dind...,https://pay2igr.igrmaharashtra.gov.in/eDisplay...,66-Notice of Lis Pendency__$__cum. Andheri 7__...
4,4,5,3551,65-Mistake Correction Letter,with D.N. Andheri 7,27/02/2023,1) Gunjan Yogeet Kapoor\n2) Tushar Subhash Oberoi,1) Shama Subhash Oberoi,"1) House No: 201, Mala No: 2, Building Boat: O...",https://pay2igr.igrmaharashtra.gov.in/eDisplay...,65-Mistake Correction Letter__$__with D.N. And...
...,...,...,...,...,...,...,...,...,...,...,...
1204,1204,1205,3173,Live Ad Licenses,Co D.N. ANDHERI 7,20/02/2023,1) COELHO NOALA,1) MUHAMMAD KUNHI K C,"1) FLAT NO:35/A, MLA NO:1ST FLOOR, BUILDING NO...",https://pay2igr.igrmaharashtra.gov.in/eDisplay...,Live Ad Licenses__$__Co D.N. ANDHERI 7__$__1) ...
1205,1205,1206,2531,Agreement,Co.D. Andheri 7,13/02/2023,"1) Chandresh Mehta, Director of Keystone Realt...",1) Rashmi Satish Kewalramani.\n2) Varun Satish...,"1) Other Information: Real Estate Project-\""Ru...",https://pay2igr.igrmaharashtra.gov.in/eDisplay...,Agreement__$__Co.D. Andheri 7__$__1) Chandresh...
1206,1206,1207,2863,Live Ad Licenses,Co D.N. Andheri 7,15/02/2023,1) Faisal Abdul Rahim Ghori,1) Jeram Chamadia (HUF) Laxman,"1) Flat No:Garage No.1, Floor No:Ground, Build...",https://pay2igr.igrmaharashtra.gov.in/eDisplay...,Live Ad Licenses__$__Co D.N. Andheri 7__$__1) ...
1207,1207,1208,2506,Release Deed,Co Dn. Andheri 7,10/02/2023,1) Monisha Rahim Harjee alias Monisha B.Charan...,1) Karim B. Charaniya,"1) House No: 1201, Floor No: Barawa Majla, Bui...",https://pay2igr.igrmaharashtra.gov.in/eDisplay...,Release Deed__$__Co Dn. Andheri 7__$__1) Monis...


In [10]:
# del os.environ['PYSPARK_PYTHON'] 
# del os.environ['PYSPARK_DRIVER_PYTHON'] 
os.environ['PYSPARK_PYTHON'] = sys.executable 
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

In [72]:
spark.stop()
del spark

In [1]:
# Approach Via Spark
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
# spark = SparkSession.builder \
#         .config('spark.executor.instances', 4) \
#         .config("spark.sql.execution.arrow.enabled", "true") \
#         .config("spark.dynamicAllocation.enabled", "true") \
#         .config("spark.dynamicAllocation.minExecutors","1") \
#         .config("spark.dynamicAllocation.maxExecutors","5") \
#         .config("spark.executor.cores",6) \
#         .config("spark.executor.memory","2g") \
#         .getOrCreate()
# spark.conf.set("spark.sql.execution.arrow.enabled", "true")
# #spark.conf.set("spark.dynamicAllocation.enabled", "true")
# spark.conf.set("spark.executor.cores", 4)
# spark.conf.set("spark.dynamicAllocation.minExecutors","1")
# spark.conf.set("spark.dynamicAllocation.maxExecutors","5")
# conf = spark.sparkContext.getConf()
spark

24/02/14 02:06:54 WARN Utils: Your hostname, ROG-Strix-G5 resolves to a loopback address: 127.0.1.1; using 192.168.29.223 instead (on interface wlp3s0)
24/02/14 02:06:54 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/02/14 02:06:54 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [110]:
del spark_df

In [6]:
#Get the Data from SQL directly
from sqlalchemy import create_engine
import pandas as pd
host = '127.0.0.1'
port = '5432'
database = 'propreturns'
username = 'postgres'
password = 'root'
url = f"jdbc:postgresql://{host}:{port}/{database}"
table_name = 'record_details_raw'

engine = create_engine(f'postgresql://{username}:{password}@{host}:{port}/{database}')

spark_df = spark.read.format("jdbc") \
        .option("url", url) \
        .option("driver", "org.postgresql.Driver") \
        .option("dbtable", table_name) \
        .option("user", username) \
        .option("password", password).load()

records_raw = pd.read_sql('record_details_raw',engine)

/tmp/ipykernel_49777/1572358846.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [7]:
spark_df.show()

+--------+---------+--------------------+------------------+----------+--------------------+--------------------+--------------------+--------------------+
|अनु क्र.|दस्त क्र.|         दस्त प्रकार|  दू. नि. कार्यालय|      वर्ष|         लिहून देणार|         लिहून घेणार|          इतर माहीती|         सूची क्र. २|
+--------+---------+--------------------+------------------+----------+--------------------+--------------------+--------------------+--------------------+
|       1|     4286|       विकसनकरारनामा|सह दु.नि. अंधेरी 7|10/03/2023|1) अजित सिंह करता...|1) नाविश रियल्टी ...|1) इतर माहिती: जम...|https://pay2igr.i...|
|       2|    13217|           भाडेपट्टा|सह दु.नि. अंधेरी 7|25/07/2023|1) श्रीमती चंद्रक...|1) महाराष्ट़ हाऊस...|1) इतर माहिती: पि...|https://pay2igr.i...|
|       3|     2449|             सेल डीड|सह दु.नि. अंधेरी 7|09/02/2023|1) नूपुर अनिल कळक...|1) हर्ष सतपाल मल्...|1) इतर माहिती: सद...|https://pay2igr.i...|
|       4|     8691|66-नोटीस ऑफ़ लिस प...|सह दु.नि. अंधेरी 7|22/0

In [13]:
#Functions
import pyspark.sql.functions as f
from pyspark.sql.types import IntegerType, StringType
from deep_translator import GoogleTranslator
from googletrans import Translator

def translator(s,source):
    return GoogleTranslator(source=source, target='en').translate(s)


def translator_v2(s,source='auto'):
        translator = Translator()
        return translator.translate(s, src='hi',dest='en').text if s != '' else 'null' 

translate_udf = f.udf(translator_v2,StringType())

new_names = []
for col in spark_df.columns:
    new_names.append(translator(col,source='auto'))

new_names = [x.replace('.','') for x in new_names]
spark_df = spark_df.toDF(*new_names)
spark_df = spark_df.withColumnRenamed(existing='Will write',new='buyer_name')
spark_df = spark_df.withColumnRenamed(existing='Will write down',new='seller_name')

In [14]:
spark_df.show()

+-----+-----------+--------------------+------------------+----------+--------------------+--------------------+--------------------+--------------------+
|Sl no|Diarrhea no|       diarrhea type|Du Prohibit Office|      Year|          buyer_name|         seller_name|   Other information|           List no 2|
+-----+-----------+--------------------+------------------+----------+--------------------+--------------------+--------------------+--------------------+
|    1|       4286|       विकसनकरारनामा|सह दु.नि. अंधेरी 7|10/03/2023|1) अजित सिंह करता...|1) नाविश रियल्टी ...|1) इतर माहिती: जम...|https://pay2igr.i...|
|    2|      13217|           भाडेपट्टा|सह दु.नि. अंधेरी 7|25/07/2023|1) श्रीमती चंद्रक...|1) महाराष्ट़ हाऊस...|1) इतर माहिती: पि...|https://pay2igr.i...|
|    3|       2449|             सेल डीड|सह दु.नि. अंधेरी 7|09/02/2023|1) नूपुर अनिल कळक...|1) हर्ष सतपाल मल्...|1) इतर माहिती: सद...|https://pay2igr.i...|
|    4|       8691|66-नोटीस ऑफ़ लिस प...|सह दु.नि. अंधेरी 7|22/05/2023|

In [15]:
spark_df.columns

['Sl no',
 'Diarrhea no',
 'diarrhea type',
 'Du Prohibit Office',
 'Year',
 'buyer_name',
 'seller_name',
 'Other information',
 'List no 2']

In [17]:
spark_df = spark_df.withColumn("to_translate",f.concat(f.col("diarrhea type"),f.lit('$'),
                                          f.col('Du Prohibit Office'),f.lit('$'),
                                          f.col('buyer_name'),f.lit('$'),
                                          f.col('seller_name'),f.lit('$'),
                                          f.col('Other information')))

In [20]:
spark_df = spark_df.withColumn('to_translate',translate_udf(spark_df['to_translate']))

In [21]:
spark_df.withColumn('check',f.split(spark_df['to_translate'],'\\$').getItem(1)).select('check').show()

+--------------------+
|               check|
+--------------------+
|       co. Andheri 7|
|  cum D.N. Andheri 7|
|    Co Dn. Andheri 7|
|  cum D.N. Andheri 7|
|  cum D.N. Andheri 7|
|       co. Andheri 7|
|  सह दु.नि. अंधेरी 7|
|  cum D.N. Andheri 7|
|  cum D.N. Andheri 7|
| with D.O. Andheri 7|
|  cum D.N. Andheri 7|
|  cum D.N. Andheri 7|
|  cum D.N. Andheri 7|
|    Co Dn. Andheri 7|
|       co. Andheri 7|
|  सह दु.नि. अंधेरी 7|
|   co d.n. Andheri 7|
|    Co Dn. Andheri 7|
|  cum D.N. Andheri 7|
|     co.in Andheri 7|
+--------------------+
only showing top 20 rows



In [21]:
spark_df = spark_df.withColumn('diarrhea type',sf.split(spark_df['to_translate'],'\\$')[0])\
        .withColumn('Du Prohibit Office',sf.split(spark_df['to_translate'],'\\$')[1])\
        .withColumn('buyer_name',sf.split(spark_df['to_translate'],'\\$')[2])\
        .withColumn('seller_name',sf.split(spark_df['to_translate'],'\\$')[3])\
        .withColumn('Other information',sf.split(spark_df['to_translate'],'\\$')[4])

In [24]:
print(spark), spark

(None, <pyspark.sql.session.SparkSession at 0x7f064c1e3f10>)

In [28]:
spark_df.count()

1209

In [23]:
table_name_sink = 'record_details_translated'

spark_df.select("*").write.format("jdbc")\
    .option("url", url) \
    .option("driver", "org.postgresql.Driver") \
    .option("dbtable", table_name_sink) \
    .option("user", username) \
    .option("password", password).save()

# PDF PART

In [11]:
#Get the Data from SQL directly
host = '127.0.0.1'
port = '5432'
database = 'propReturns'
username = 'postgres'
password = 'Sunrise12345'
url = f"jdbc:postgresql://{host}:{port}/{database}"
table_name = 'record_details_translated_v2'

engine = create_engine(f'postgresql://{username}:{password}@{host}:{port}/{database}')

# spark_df = spark.read.format("jdbc") \
#         .option("url", url) \
#         .option("driver", "org.postgresql.Driver") \
#         .option("dbtable", table_name) \
#         .option("user", username) \
#         .option("password", password).load()

records_translated = pd.read_sql('record_details_translated_v2',engine)

In [14]:
records_translated.columns

Index(['index', 'Sl no', 'Diarrhea no', 'diarrhea type', 'Du Prohibit Office',
       'Year', 'buyer_name', 'seller_name', 'Other information', 'List no 2'],
      dtype='object')

In [4]:
import urllib.request
import ssl

# This restores the same behavior as before.
context = ssl._create_unverified_context()
# urllib.urlopen("https://no-valid-cert", context=context)
pdf_path = "https://pay2igr.igrmaharashtra.gov.in/eDisplay/propertydetails/indexii/Mw%3D%3D/MDUxNDA1MzY5OTAwMDAwNDI4NjIwMjNJUw%3D%3D"
def download_file(download_url, filename):
    response = urllib.request.urlopen(download_url,context=context)    
    file = open(filename + ".pdf", 'wb')
    file.write(response.read())
    file.close()
 
download_file(download_url=pdf_path, filename="Test")

In [18]:
#Approach Via Pandas --> Just downloading the files
batch_size = 100
start = 0
end = len(records_translated)
# end = 5
os.makedirs("pdfs", exist_ok = True)
for batch in range(start,end,batch_size):
    print(f'Batch -> [{batch},{batch+batch_size}]')
    temp_df = records_translated.iloc[batch:batch+batch_size,:].copy()
    for index, row in tqdm(temp_df.iterrows()):
        download_file(download_url=row['List no 2'], filename=f"pdfs/{row['Sl no']}")

Batch -> [0,100]


100it [01:39,  1.01it/s]


Batch -> [100,200]


100it [01:45,  1.06s/it]


Batch -> [200,300]


...

100it [01:39,  1.01it/s]


Batch -> [1100,1200]


100it [01:34,  1.06it/s]


Batch -> [1200,1300]


9it [00:08,  1.04it/s]


In [13]:
def curves_to_edges(cs):
    edges = []
    for c in cs:
        edges += pdfplumber.utils.rect_to_edges(c)
    return edges

# Get the bounding boxes of the tables on the page.
def not_within_bboxes(obj):
    """Check if the object is in any of the table's bbox."""
    def obj_in_bbox(_bbox):
        """See https://github.com/jsvine/pdfplumber/blob/stable/pdfplumber/table.py#L404"""
        v_mid = (obj["top"] + obj["bottom"]) / 2
        h_mid = (obj["x0"] + obj["x1"]) / 2
        x0, top, x1, bottom = _bbox
        return (h_mid >= x0) and (h_mid < x1) and (v_mid >= top) and (v_mid < bottom)
    return not any(obj_in_bbox(__bbox) for __bbox in bboxes)


In [33]:
save_path = r'C:\Users\ayush\OneDrive\Desktop\Projects\propreturns_assignment\Mumbai_Real_Estate_Data\pdfs\1.pdf'
pdf = pdfplumber.open(save_path)
page = pdf.pages[0]

table_settings = {
    "vertical_strategy": "explicit",
    "horizontal_strategy": "explicit",
    "explicit_vertical_lines": curves_to_edges(page.curves + page.edges),
    "explicit_horizontal_lines": curves_to_edges(page.curves + page.edges),
    "intersection_y_tolerance": 10,
}

bboxes = [table.bbox for table in page.find_tables(table_settings=table_settings)]

non_table_data = page.filter(not_within_bboxes).extract_text()

table_data_in_nested_list = page.extract_table()
cols = ["sl_no", "pdf_urls", "save_path", "non_table_text", "table_dict"]
extracted_df = pd.DataFrame(columns = cols)
table_data = {li[0]: li[1] for li in table_data_in_nested_list}
curr_df = pd.DataFrame([[0, '', '', non_table_data, table_data]], columns = cols)
extracted_df = pd.concat([extracted_df, curr_df],ignore_index=True)

In [116]:
extracted_df_2 = extracted_df.copy()

In [111]:
def translate_table(non_table_data, table_data):
    visited = False
    temp_dict = {}
    temp_string = ''
    for key, value in table_data.items():
        combined_value = f'{key}__$__{value}' if visited else f'{key}__$__{value}__$__{non_table_data}'
        combined_value = translator_v2(combined_value).split('__$__')
        if not visited:
            visited = True
            temp_string = combined_value[2]
        temp_dict[combined_value[0]] = combined_value[1]
        
    return pd.Series([temp_dict, temp_string])

In [112]:
# extracted_df_2[["table_dict", "non_table_text"]] = 
extracted_df_2[["table_dict", "non_table_text"]].apply(lambda x: translate_table(x["non_table_text"], x["table_dict"]),axis=1)

,0,1
0,{'(1)Subrogation type': 'Development agreement...,4286514 List  . 2 Second Director: Co-Dr. And...


In [117]:
extracted_df_2 = pd.concat([extracted_df_2, extracted_df_2[["table_dict", "non_table_text"]].apply(lambda x: translate_table(x["non_table_text"], x["table_dict"]),axis=1)], axis = 1)

In [124]:
extracted_df_2 = extracted_df_2.rename(columns = {0: "table_data", 1: "non_table_data"})

In [125]:
extracted_df_2

,sl_no,pdf_urls,save_path,non_table_text,table_dict,table_data,non_table_data
0,0,,,4286514 सचू ी  . २ दुयम िनबंधक :सह दु.िन. अं...,"{'(1)दतऐवज कार': 'िवकसनकरारनामा', '(2)मोबदला...",{'(1)Substitute type': 'Development agreement'...,4286514 list . 2 Second Director: Co-Dr. Andhe...


In [43]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
from deep_translator import GoogleTranslator
from googletrans import Translator
from sqlalchemy import create_engine
from tqdm import tqdm

#Functions
def translator(s,source):
    return GoogleTranslator(source=source, target='en').translate(s)


def translator_v2(s,source='auto'):
        translator = Translator()   
        return translator.translate(s, src='hi',dest='en').text if s != '' else 'null' 

# translate_udf = sf.udf(translator_v2,StringType())

db_user = 'postgres'
db_password = 'root'
db_host = '127.0.0.1'
db_port = '5432'
db_name = 'propreturns'
table_name = 'record_details_raw'
engine = create_engine(f'postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}')

records_raw = pd.read_sql(table_name,engine)
print(f'Table {table_name} loaded. Rows: {len(records_raw)}')
records_translate = records_raw.copy()
records_translate.columns = [translator(col,source='auto') for col in records_translate.columns]
cols_to_translate = [
    'diarrhea type',
    'Du. Prohibit. Office',
    'Will write', 
    'Will write down', 
    'Other information'
    ]

records_translate['to_translate'] = records_translate['diarrhea type'] + '__$__' \
                                    + records_translate['Du. Prohibit. Office'] + '__$__' \
                                    + records_translate['Will write'] + '__$__' \
                                    + records_translate['Will write down'] + '__$__' \
                                    + records_translate['Other information']  

records_translated = pd.DataFrame(columns=records_translate.columns)
batch_size = 1
start = 0

Table record_details_raw loaded. Rows: 1209


In [44]:
for i in tqdm(range(start,len(records_translate))):
    print(i)

100%|██████████| 1209/1209 [00:00<00:00, 150138.08it/s]

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [45]:
for batch in tqdm(range(start,len(records_translate))):
    temp_df = records_translate.iloc[batch:batch+batch_size,:].copy()
    temp_df['to_translate'] = temp_df['to_translate'].apply(translator_v2)
    records_translated = pd.concat([records_translated,temp_df])
records_translated.rename(columns={'Will write':'buyer_name','Will write down':'seller_name'},inplace=True)

  1%|▏         | 16/1209 [00:10<13:05,  1.52it/s]


KeyboardInterrupt: 

# KAFKA PART

## Random

In [2]:
from confluent_kafka.admin import AdminClient, NewTopic


admin_client = AdminClient({
    "bootstrap.servers": "localhost:9092"
})

topic_list = []
topic_list.append(NewTopic("example_topic", 1, 1))
admin_client.create_topics(topic_list)

{'example_topic': <Future at 0x7f1252f776a0 state=running>}

In [10]:
admin_client.list_topics().topics['example_topic']

TopicMetadata(example_topic, 1 partitions)

In [19]:
records_raw_topic = NewTopic(topic='records_raw_topic',num_partitions=3,replication_factor=1)
records_raw_topic

NewTopic(topic=records_raw_topic,num_partitions=3)

In [21]:
admin_client.create_topics([records_raw_topic])

{'records_raw_topic': <Future at 0x7f1250722da0 state=running>}

In [ ]:
import time
import json
from uuid import uuid4
from confluent_kafka import Producer

jsonString1 = """ {"name":"Gal", "email":"Gadot84@gmail.com", "salary": "8345.55"} """
jsonString2 = """ {"name":"Dwayne", "email":"Johnson52@gmail.com", "salary": "7345.75"} """
jsonString3 = """ {"name":"Momoa", "email":"Jason91@gmail.com", "salary": "3345.25"} """

jsonv1 = jsonString1.encode()
jsonv2 = jsonString2.encode()
jsonv3 = jsonString3.encode()

def delivery_report(errmsg, msg):
	"""
	Reports the Failure or Success of a message delivery.
	Args:
		errmsg (KafkaError): The Error that occurred while message producing.
		msg (Actual message): The message that was produced.
	Note:
		In the delivery report callback the Message.key() and Message.value()
		will be the binary format as encoded by any configured Serializers and
		not the same object that was passed to produce().
		If you wish to pass the original object(s) for key and value to delivery
		report callback we recommend a bound callback or lambda where you pass
		the objects along.
	"""
	if errmsg is not None:
		print("Delivery failed for Message: {} : {}".format(msg.key(), errmsg))
		return
	print('Message: {} successfully produced to Topic: {} Partition: [{}] at offset {}'.format(
		msg.key(), msg.topic(), msg.partition(), msg.offset()))

kafka_topic_name = "kf.topic.empdev"
#Change your Kafka Topic Name here. For this Example: lets assume our Kafka Topic has 3 Partitions==> 0,1,2
#And We are producing messages uniformly to all partitions.
#We are sending the message as ByteArray.
#If We want read the same message from a Java Consumer Program 
#We can configure KEY_DESERIALIZER_CLASS_CONFIG = ByteArrayDeserializer.class
# and VALUE_DESERIALIZER_CLASS_CONFIG = ByteArrayDeserializer.class

mysecret = "yourjksPassword"
#you can call remote API to get JKS password instead of hardcoding like above

print("Starting Kafka Producer") 
conf = {
		'bootstrap.servers' : 'm1.msk.us-east.aws.com:9094, m2.msk.us-east.aws.com:9094, m3.msk.us-east.aws.com:9094',
		'security.protocol' : 'SSL',
		'ssl.keystore.password' : mysecret,
		'ssl.keystore.location' : './certkey.p12'
		}
		
print("connecting to Kafka topic...")
producer1 = Producer(conf)

# Trigger any available delivery report callbacks from previous produce() calls
producer1.poll(0)

try:
	# Asynchronously produce a message, the delivery report callback
	# will be triggered from poll() above, or flush() below, when the message has
	# been successfully delivered or failed permanently.
	producer1.produce(topic=kafka_topic_name, key=str(uuid4()), value=jsonv1, on_delivery=delivery_report)
	producer1.produce(topic=kafka_topic_name, key=str(uuid4()), value=jsonv2, on_delivery=delivery_report)
	producer1.produce(topic=kafka_topic_name, key=str(uuid4()), value=jsonv3, on_delivery=delivery_report)
	
	# Wait for any outstanding messages to be delivered and delivery report
	# callbacks to be triggered.
	producer1.flush()
	
except Exception as ex:
	print("Exception happened :",ex)
	
print("\n Stopping Kafka Producer")


In [33]:
from confluent_kafka import Producer, Consumer

def delivery_report(errmsg, msg):
	if errmsg is not None:
		print(f"Delivery failed for Message: {msg.key()} : {errmsg}")
		return
	print(f'Message: {msg.key()} successfully produced to Topic: {msg.topic()} Partition: [{msg.partition()}] at offset {msg.offset()}')

In [27]:
from uuid import uuid4
from confluent_kafka.admin import AdminClient, NewTopic
kafka_topic_name = "records_raw_topic"
print("Starting Kafka Producer") 
conf = {
		'bootstrap.servers' : "localhost:9092",
		# 'security.protocol' : 'SSL',
		# 'ssl.keystore.password' : mysecret,
		# 'ssl.keystore.location' : './certkey.p12'
		}
		
print("connecting to Kafka topic...")
producerScrapper = Producer(conf)

# Trigger any available delivery report callbacks from previous produce() calls
producerScrapper.poll(0)

jsonString1 = """ {"name":"Gal", "email":"Gadot84@gmail.com", "salary": "8345.55"} """
jsonString2 = """ {"name":"Dwayne", "email":"Johnson52@gmail.com", "salary": "7345.75"} """
jsonString3 = """ {"name":"Momoa", "email":"Jason91@gmail.com", "salary": "3345.25"} """

jsonv1 = jsonString1.encode()
jsonv2 = jsonString2.encode()
jsonv3 = jsonString3.encode()

try:
	# Asynchronously produce a message, the delivery report callback
	# will be triggered from poll() above, or flush() below, when the message has
	# been successfully delivered or failed permanently.
	producerScrapper.produce(topic=kafka_topic_name, key=str(uuid4()), value=jsonv1, on_delivery=delivery_report)
	producerScrapper.produce(topic=kafka_topic_name, key=str(uuid4()), value=jsonv2, on_delivery=delivery_report)
	producerScrapper.produce(topic=kafka_topic_name, key=str(uuid4()), value=jsonv3, on_delivery=delivery_report)
	
	# Wait for any outstanding messages to be delivered and delivery report
	# callbacks to be triggered.
	producerScrapper.flush()
	
except Exception as ex:
	print("Exception happened :",ex)
	
print("\n Stopping Kafka Producer")


Starting Kafka Producer
connecting to Kafka topic...
Message: b'b86e6982-1550-4ae2-947d-691f0e3913fb' successfully produced to Topic: records_raw_topic Partition: [1] at offset 2
Message: b'0ec061ab-4b00-473a-9d3f-4d12ba3436ce' successfully produced to Topic: records_raw_topic Partition: [0] at offset 0
Message: b'9fed24bc-83cb-4ccd-a948-1ad60deb2122' successfully produced to Topic: records_raw_topic Partition: [0] at offset 1

 Stopping Kafka Producer


In [31]:
admin_client.list_topics().topics['records_raw_topic']

TopicMetadata(records_raw_topic, 3 partitions)

In [35]:
conf = {'bootstrap.servers': 'localhost:9092',
        'group.id': 'foo',
        'auto.offset.reset': 'smallest'}
consumerScrapper = Consumer(conf)
consumerScrapper.subscribe()

## Topic Creation and Producers

In [17]:
#Importing Libraries
from confluent_kafka import Producer, Consumer
from confluent_kafka.admin import AdminClient, NewTopic
from pyspark.sql import SparkSession
import pyspark.sql.functions as f
import pyspark.sql.types as t
import json


In [13]:
def delivery_report(errmsg, msg):
	if errmsg is not None:
		print(f"Delivery failed for Message: {msg.key()} : {errmsg}")
		return
	print(f'Message: {msg.key()} successfully produced to Topic: {msg.topic()} Partition: [{msg.partition()}] at offset {msg.offset()}')

In [2]:
#Initialization
conf = {
    "bootstrap.servers": "localhost:9092"
}
spark_df.toJSON().first()
admin_client = AdminClient(conf=conf)
spark = SparkSession.builder.getOrCreate()
spark

24/02/15 14:45:01 WARN Utils: Your hostname, ayushbajpayee-ThinkPad-T480 resolves to a loopback address: 127.0.1.1; using 192.168.1.20 instead (on interface wlp3s0)
24/02/15 14:45:01 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/02/15 14:45:02 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
#Create Topic
topic_name = 'records_raw_topic'
records_raw_topic = NewTopic(topic=topic_name,num_partitions=1,replication_factor=1)

if topic_name not in admin_client.list_topics().topics.keys():
    admin_client.create_topics([records_raw_topic])
    print('Topic Created')

else:
    print('Topic already present, skipping creation')

Topic Created


In [4]:
#Get Spark Dataframe from postgres localhost
host = 'localhost'
port = '5432'
database = 'propreturns'
url = f"jdbc:postgresql://{host}:{port}/{database}"
properties = {
    "user": "postgres",
    "password": "root",
    "driver": "org.postgresql.Driver"
}

# Define the table you want to read
table_name = "record_details_translated"

# Read data from PostgreSQL into a Spark DataFrame
spark_df = spark.read.jdbc(url=url, table=table_name, properties=properties)

# Show the DataFrame
spark_df.show()

+-----+-----------+--------------------+--------------------+----------+--------------------+--------------------+--------------------+--------------------+--------------------+
|Sl no|Diarrhea no|       diarrhea type|  Du Prohibit Office|      Year|          buyer_name|         seller_name|   Other information|           List no 2|        to_translate|
+-----+-----------+--------------------+--------------------+----------+--------------------+--------------------+--------------------+--------------------+--------------------+
|    1|      13217|          Rent Lease|  cum D.N. Andheri 7|25/07/2023|1) Bhavin Sheth, ...|1) Hanmant Dhanva...|1) Other informat...|https://pay2igr.i...|Rent Lease$cum D....|
|    2|       4286|Development agree...|       co. Andheri 7|10/03/2023|1) Ajit Singh Kar...|1) Paresh Ranchho...|1) Other Informat...|https://pay2igr.i...|Development agree...|
|    3|       2449|           Sale Deed|   cum Dn. Andheri 7|09/02/2023|1) By Nupur Anil ...|1) Harsh Satpal M

In [14]:
sample_json = json.loads(spark_df.toJSON().first())
sample_json

{'Sl no': '1',
 'Diarrhea no': '13217',
 'diarrhea type': 'Rent Lease',
 'Du Prohibit Office': 'cum D.N. Andheri 7',
 'Year': '25/07/2023',
 'buyer_name': '1) Bhavin Sheth, Authorized Trustee on behalf of Smt. Chandrakaladevi Somani Charity Trust',
 'seller_name': '1) Hanmant Dhanvare, Executive Engineer, Maharashtra Housing and Area Development Authority, Mumbai Housing and Area Development Board, Mumbai (suit to remain present in the office as per Section 88 of Non-Debit Act 1908)\n2) Neelima Dhaygude, Joint Chief Officer, Maharashtra Housing and Area Development Authority, Mumbai Housing and Area Development Board, Mumbai (Suite to attend office as per Section 88 of Non-Registration Act 1908)',
 'Other information': '1) Other information: Piece or parcel of land measuring 1140.04 square meters bearing. CTS no. A/791 Bandra Reclamation ADJ/1100900/532/2023 Bharlele Stamp duty Rs 3400630 dated 8/7/2023 Bandra West 400050 Other information documents as mentioned.',
 'List no 2': 'https

In [20]:
type(sample_json)

dict

In [46]:
#imports
from selenium import webdriver 
from selenium.webdriver.support.ui import Select 
from selenium.webdriver.common.by import By
import pandas as pd
from sqlalchemy import create_engine
from tqdm import tqdm
from time import sleep
import warnings
import psycopg2
warnings.filterwarnings('ignore')

input_params_dropdown_by_id = {}
input_params_dropdown_by_id['dbselect'] = '2023' #Select Year
input_params_dropdown_by_id['district_id'] = 'मुंबई उपनगर' #District
input_params_dropdown_by_id['taluka_id'] = 'अंधेरी' #Taluka
input_params_dropdown_by_id['village_id'] = 'बांद्रा' #Village

input_params_text_by_id = {}
input_params_text_by_id['free_text'] = '2023' 

url = 'https://pay2igr.igrmaharashtra.gov.in/eDisplay/Propertydetails/index' 
driver = webdriver.Chrome() 
driver.implicitly_wait(10)
driver.get(url)

for key, value in input_params_dropdown_by_id.items():
    print(key,value)
    dropdown = driver.find_element(by=By.ID,value=key)
    dropdown_select = Select(dropdown)
    for option in dropdown_select.options:
        if option.text == value:
            option.click()
            print(f'Clicked {option.text} for {key}')
            break
    driver.implicitly_wait(2)

#Input reg year
driver.find_element(by=By.ID, value='free_text').send_keys(input_params_text_by_id['free_text'])
driver.implicitly_wait(2)

#Input Captcha
print('Enter Captcha in the box [DO NOT PRESS ENTER]')
sleep(10)

#Submit to get result set
driver.find_element(by=By.ID,value='submit').click()
sleep(10)


#Click on all pages
dropdown = driver.find_element(by=By.NAME, value='tableparty_length')
dropdown_select = Select(dropdown)
for option in dropdown_select.options:
    if option.text == 'All':
        option.click()
        print('Set to All Records')
        break

print('Table Loaded')
#scrap records
records_raw = pd.DataFrame(columns=['अनु क्र.','दस्त क्र.','दस्त प्रकार','दू. नि. कार्यालय','वर्ष','लिहून देणार','लिहून घेणार','इतर माहीती','सूची क्र. २'])

print('Data Loading...')
sleep(10)

dbselect 2023
Clicked 2023 for dbselect
district_id मुंबई उपनगर
Clicked मुंबई उपनगर for district_id
taluka_id अंधेरी
Clicked अंधेरी for taluka_id
village_id बांद्रा
Clicked बांद्रा for village_id
Enter Captcha in the box [DO NOT PRESS ENTER]
Set to All Records
Table Loaded
Data Loading...


In [49]:
# for index, table in enumerate(tqdm(driver.find_elements(by=By.ID, value='tbdata'))):
sample_dict = {}
producerScrapper = Producer(conf)

for table in tqdm(driver.find_elements(by=By.ID, value='tbdata')):
    data = [item.text if item.text != 'सूची क्र. २' else item.find_element(by=By.TAG_NAME,value='a').get_attribute('href') for item in table.find_elements(by=By.XPATH, value=".//*[self::td or self::th]")]
    sample_dict['अनु क्र.'] = data[0]
    sample_dict['दस्त क्र.'] = data[1]
    sample_dict['दस्त प्रकार'] = data[2]
    sample_dict['दू. नि. कार्यालय'] = data[3]
    sample_dict['वर्ष'] = data[4]
    sample_dict['लिहून देणार'] = data[5]
    sample_dict['लिहून घेणार'] = data[6]
    sample_dict['इतर माहीती'] = data[7]
    sample_dict['सूची क्र. २'] = data[8]
    producerScrapper.produce(topic=topic_name, key=sample_dict['अनु क्र.'], value=json.dumps(sample_dict).encode(), on_delivery=delivery_report)

producerScrapper.flush()

100%|██████████| 1209/1209 [03:43<00:00,  5.41it/s]

Message: b'1' successfully produced to Topic: records_raw_topic Partition: [0] at offset 30
Message: b'2' successfully produced to Topic: records_raw_topic Partition: [0] at offset 31
Message: b'3' successfully produced to Topic: records_raw_topic Partition: [0] at offset 32
Message: b'4' successfully produced to Topic: records_raw_topic Partition: [0] at offset 33
Message: b'5' successfully produced to Topic: records_raw_topic Partition: [0] at offset 34
Message: b'6' successfully produced to Topic: records_raw_topic Partition: [0] at offset 35
Message: b'7' successfully produced to Topic: records_raw_topic Partition: [0] at offset 36
Message: b'8' successfully produced to Topic: records_raw_topic Partition: [0] at offset 37
Message: b'9' successfully produced to Topic: records_raw_topic Partition: [0] at offset 38
Message: b'10' successfully produced to Topic: records_raw_topic Partition: [0] at offset 39
Message: b'11' successfully produced to Topic: records_raw_topic Partition: [0]

0

In [52]:
admin_client.list_topics().topics.keys()

dict_keys(['_confluent-controlcenter-7-5-3-1-MonitoringMessageAggregatorWindows-THREE_HOURS-changelog', 'default_ksql_processing_log', '_confluent-monitoring', 'connect-offsets', '__consumer_offsets', '_confluent-controlcenter-7-5-3-1-aggregatedTopicPartitionTableWindows-ONE_MINUTE-repartition', '_confluent_balancer_api_state', '_confluent-controlcenter-7-5-3-1-expected-group-consumption-rekey', '_confluent-controlcenter-7-5-3-1-aggregate-topic-partition-store-changelog', '_confluent-controlcenter-7-5-3-1-aggregate-topic-partition-store-repartition', '_confluent-controlcenter-7-5-3-1-TriggerActionsStore-changelog', '_confluent-controlcenter-7-5-3-1-monitoring-aggregate-rekey-store-repartition', '__transaction_state', '_confluent-controlcenter-7-5-3-1-aggregatedTopicPartitionTableWindows-ONE_MINUTE-changelog', '_confluent-controlcenter-7-5-3-1-Group-ONE_MINUTE-repartition', '_confluent-controlcenter-7-5-3-1-MonitoringMessageAggregatorWindows-THREE_HOURS-repartition', '_confluent-control

### Topic Creation reading from postgres table

In [8]:
# Topic Creation reading from postgres table
import warnings
warnings.filterwarnings('ignore')
from confluent_kafka.admin import AdminClient, NewTopic
from confluent_kafka import Producer
from pyspark.sql import SparkSession

In [5]:
spark = SparkSession.Builder().getOrCreate()
spark

24/02/20 13:57:05 WARN Utils: Your hostname, ayushbajpayee-ThinkPad-T480 resolves to a loopback address: 127.0.1.1; using 192.168.1.20 instead (on interface wlp3s0)
24/02/20 13:57:05 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/02/20 13:57:05 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [15]:
# Define JDBC connection properties
jdbc_url = "jdbc:postgresql://localhost:5432/propreturns"
connection_properties = {
    "user": "postgres",
    "password": "root",
    "driver": "org.postgresql.Driver"
}

# Define the table name
table_name = "record_details_raw"

# Read data from PostgreSQL table as a DataFrame
spark_df = spark \
    .read \
    .jdbc(url=jdbc_url, table=table_name, properties=connection_properties)

# Perform further processing or analysis on the DataFrame
# For example, you can show the data
spark_df.show()

+-----+--------+---------+--------------------+------------------+----------+--------------------+--------------------+--------------------+--------------------+
|index|अनु क्र.|दस्त क्र.|         दस्त प्रकार|  दू. नि. कार्यालय|      वर्ष|         लिहून देणार|         लिहून घेणार|          इतर माहीती|         सूची क्र. २|
+-----+--------+---------+--------------------+------------------+----------+--------------------+--------------------+--------------------+--------------------+
|    0|       1|     4286|       विकसनकरारनामा|सह दु.नि. अंधेरी 7|10/03/2023|1) अजित सिंह करता...|1) नाविश रियल्टी ...|1) इतर माहिती: जम...|https://pay2igr.i...|
|    1|       2|    13217|           भाडेपट्टा|सह दु.नि. अंधेरी 7|25/07/2023|1) श्रीमती चंद्रक...|1) महाराष्ट़ हाऊस...|1) इतर माहिती: पि...|https://pay2igr.i...|
|    2|       3|     2449|             सेल डीड|सह दु.नि. अंधेरी 7|09/02/2023|1) नूपुर अनिल कळक...|1) हर्ष सतपाल मल्...|1) इतर माहिती: सद...|https://pay2igr.i...|
|    3|       4|     8691|66

In [35]:
from utils.dataLoader import postgresToSpark, postgresToPandas

ImportError: cannot import name 'postgresToPandas' from 'utils.dataLoader' (/home/ayushbajpayee/Desktop/projects/Mumbai_Real_Estate_Data/utils/dataLoader.py)

In [36]:
postgresToPandas(table_name='record_details_translated')

NameError: name 'postgresToPandas' is not defined

In [14]:
#Create Topic
admin_client = AdminClient({'bootstrap.servers':'localhost:9092'})
topic_name = 'records_raw_topic'
records_raw_topic = NewTopic(topic=topic_name,num_partitions=1,replication_factor=1)

if topic_name not in admin_client.list_topics().topics.keys():
    admin_client.create_topics([records_raw_topic])
    print('Topic Created')

else:
    choice = int(input('Topic already present, Preserve/Delete and create [0/1]: '))
    if choice:
        admin_client.delete_topics([records_raw_topic])
        admin_client.create_topics([records_raw_topic])
        print('Topic Re Created!')
    else:
        print('Topic Preserved!')

Topic Re Created!


In [18]:
spark_df.printSchema()

root
 |-- index: long (nullable = true)
 |-- अनु क्र.: string (nullable = true)
 |-- दस्त क्र.: string (nullable = true)
 |-- दस्त प्रकार: string (nullable = true)
 |-- दू. नि. कार्यालय: string (nullable = true)
 |-- वर्ष: string (nullable = true)
 |-- लिहून देणार: string (nullable = true)
 |-- लिहून घेणार: string (nullable = true)
 |-- इतर माहीती: string (nullable = true)
 |-- सूची क्र. २: string (nullable = true)



In [27]:
spark_df.select('अनु क्र.')

AnalysisException: Syntax error in attribute name: अनु क्र..

In [29]:
kafka_params = {
    "kafka.bootstrap.servers": "localhost:9092",
    "topic": topic_name
}

# Write DataFrame to Kafka topic
query = spark_df \
    .selectExpr("CAST(index AS STRING)", "to_json(struct(*)) AS value") \
    .writeStream \
    .format("kafka") \
    .options(**kafka_params) \
    .start()

# Wait for the query to terminate
query.awaitTermination()


AnalysisException: [WRITE_STREAM_NOT_ALLOWED] `writeStream` can be called only on streaming Dataset/DataFrame.

## Kafka Consumer

In [1]:
import warnings
warnings.filterwarnings('ignore')
from pyspark.sql import SparkSession
import pyspark.sql.functions as f
import pyspark.sql.types as t
from confluent_kafka import Consumer

In [67]:
# del consumer

In [71]:
conf = {'bootstrap.servers': 'localhost:9092',
        'group.id':'fo'}
consumer = Consumer(conf)

In [76]:
topic = 'records_raw_topic'
consumer.subscribe([topic])

In [77]:
try:
    while True:
        msg = consumer.poll(1.0)
        if msg is None:
            print('Waiting....')

        elif msg.error():
            print(f'ERROR: {msg.error()}')

        else:
            print(f"Consumed Event from topic: {msg.topic()} key: {msg.key().decode('utf-8')}, Value: {msg.value().decode('utf-8')}")

except KeyboardInterrupt:
    print('Stopped!')
    pass

finally:
    consumer.close()

Waiting....
Waiting....
Waiting....
Waiting....
Waiting....
Waiting....
Waiting....
Waiting....
Waiting....
Waiting....
Waiting....
Waiting....
Waiting....
Waiting....
Waiting....
Stopped!
